In [2]:
import torch
import numpy as np
from qmc.tracehess import autograd_trace_hessian
from torch import nn, optim

In [37]:
class NelectronVander(nn.Module):
    #ansatz given by the Vandermonde determinant of the one electron wavefunctions e^(-alpha*r_i)
    #input is 1-element tensor alpha and 1D tensor dim which determines the number of particles (i.e. the dimension)
    def __init__(self, alpha, dim):
        super(NelectronVander, self).__init__()
        self.alpha = nn.Parameter(alpha)
        self.dim = nn.Parameter(dim)
            
    def forward(self, x):
        #returns the log prob. of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return 2 * torch.sum(torch.log(torch.abs(a[...,torch.triu(torch.ones(self.dim.shape[0],self.dim.shape[0]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.dim.shape[0],self.dim.shape[0]), diagonal=1).nonzero(as_tuple = True)[1] ])),-1)
    
    
    def wave(self,x):
        # Returns the value of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return torch.prod(a[...,torch.triu(torch.ones(self.dim.shape[0],self.dim.shape[0]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.dim.shape[0],self.dim.shape[0]), diagonal=1).nonzero(as_tuple = True)[1] ],-1)
    
    
        

In [38]:
class NelectronVanderWithMult(nn.Module):
    #ansatz given by the Vandermonde determinant of the one electron wavefunctions e^(-alpha * r_i) 
    #multiplied by e^(-beta * (r_1 + r_2 + ... r_N))
    #input is 1-element tensors alpha, beta and 1D tensor dim which determines the number of particles (i.e. the dimension)
    def __init__(self, alpha, beta, dim):
        super(NelectronVanderWithMult, self).__init__()
        self.alpha = nn.Parameter(alpha)
        self.beta = nn.Parameter(beta)
        self.dim = nn.Parameter(dim)

    
    def forward(self, x):
        #returns the log prob. of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return 2 * ( -self.beta * torch.sum(x, -1)
            + torch.sum(torch.log(torch.abs(a[...,torch.triu(torch.ones(self.dim.shape[0],self.dim.shape[0]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.dim.shape[0],self.dim.shape[0]), diagonal=1).nonzero(as_tuple = True)[1] ])),-1) )
    
    
    def wave(self,x):
        # Returns the value of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return torch.exp(-self.beta * torch.sum(x, -1)) * torch.prod(a[...,torch.triu(torch.ones(self.dim.shape[0],self.dim.shape[0]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.dim.shape[0],self.dim.shape[0]), diagonal=1).nonzero(as_tuple = True)[1] ],-1)
    
    

In [39]:
f = NelectronVander(torch.rand(1),torch.ones(5))

In [40]:
f(torch.rand((6,4,5)))

tensor([[-36.5575, -36.2937, -34.5946, -35.5061],
        [-49.0331, -45.3782, -45.9216, -33.7046],
        [-51.8186, -35.8793, -39.3723, -36.9850],
        [-36.8325, -40.1819, -50.8912, -39.4910],
        [-40.7212, -32.9279, -35.5383, -37.8271],
        [-34.4723, -35.6776, -39.6079, -53.9690]], grad_fn=<MulBackward0>)

In [41]:
f.wave(torch.rand((3,7,7)))

tensor([[-8.2301e-08, -1.4820e-08, -6.7759e-09,  1.2520e-10,  6.8677e-11,
          1.3281e-08, -9.7756e-09],
        [ 8.9488e-11, -5.9632e-10, -3.7083e-09, -2.3774e-08, -1.9402e-08,
         -1.7927e-09, -1.5923e-14],
        [-8.2493e-10, -1.9931e-09,  2.5859e-08, -2.3488e-08,  1.2741e-08,
          4.5950e-09, -8.6249e-08]], grad_fn=<ProdBackward1>)

In [42]:
x = torch.rand(4,5)

In [43]:
f = NelectronVanderWithMult(torch.rand(1),torch.rand(1),torch.ones(6))

In [44]:
f(torch.rand((9,6)))

tensor([-71.3939, -73.2012, -68.9493, -71.1632, -86.7975, -76.6327, -81.3204,
        -75.6127, -73.6680], grad_fn=<MulBackward0>)

In [36]:
f.wave(torch.rand((3,7,6)))

tensor([[ 5.0057e-15,  1.3982e-15, -4.6414e-16, -2.4316e-20, -5.8542e-15,
          8.7120e-19, -1.6698e-15],
        [-1.0743e-15,  3.5532e-16,  6.0558e-16,  2.7957e-20, -5.8041e-17,
         -8.1037e-15,  2.1230e-14],
        [-4.0730e-20, -1.0970e-19, -7.0751e-16,  7.5140e-22,  1.0852e-16,
         -9.7224e-19,  7.4480e-17]], grad_fn=<MulBackward0>)

In [50]:
def test_nelectron_vander_asym():
    config_dimension = 5
    f = NelectronVander(torch.rand(1), torch.ones(config_dimension))
    x = torch.rand(1000,config_dimension)
    for i in range(1000):
        t = torch.randperm(config_dimension)[:2]
        a = torch.clone(x[i])
        a[t[0]] = x[i][t[1]]
        a[t[1]] = x[i][t[0]]
        assert torch.isclose(f.wave(a), -f.wave(x[i]))
    

In [52]:
def test_nelectron_vanderWithmult_asym():
    config_dimension = 5
    f = NelectronVanderWithMult(torch.rand(1),torch.rand(1), torch.ones(config_dimension))
    x = torch.rand(1000,config_dimension)
    for i in range(1000):
        t = torch.randperm(config_dimension)[:2]
        a = torch.clone(x[i])
        a[t[0]] = x[i][t[1]]
        a[t[1]] = x[i][t[0]]
        assert torch.isclose(f.wave(a), -f.wave(x[i]))

In [54]:
x = torch.rand(1000,config_dimension)
for i in range(1000):
    t = torch.randperm(config_dimension)[:2]
    a = torch.clone(x[i])
    a[t[0]] = x[i][t[1]]
    a[t[1]] = x[i][t[0]]
    print(torch.isclose(f.wave(a), -f.wave(x[i])))

tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)

tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)

In [81]:
x

tensor([[0.8010, 0.3007],
        [0.3570, 0.2304],
        [0.6954, 0.7135],
        [0.4398, 0.9295],
        [0.2726, 0.7223]])

In [77]:
x.shape[0]

5

In [78]:
idx = torch.randperm(x.shape[0])

In [79]:
x[idx,:]

tensor([[0.5561, 0.3615],
        [0.8880, 0.0766],
        [0.9743, 0.2396],
        [0.7600, 0.0135],
        [0.5204, 0.8210]])

In [82]:
print(x[idx,:],x)

tensor([[0.2726, 0.7223],
        [0.4398, 0.9295],
        [0.3570, 0.2304],
        [0.8010, 0.3007],
        [0.6954, 0.7135]]) tensor([[0.8010, 0.3007],
        [0.3570, 0.2304],
        [0.6954, 0.7135],
        [0.4398, 0.9295],
        [0.2726, 0.7223]])


In [49]:
config_dimension = 4
f = NelectronVander(torch.rand(1), torch.ones(config_dimension))
x = torch.rand(100,config_dimension)
for i in range(100):
    t = torch.randperm(config_dimension)[:2]
    a = torch.clone(x[i])
    a[t[0]] = x[i][t[1]]
    a[t[1]] = x[i][t[0]]
    assert torch.isclose(f.wave(a), -f.wave(x[i]))


In [258]:
f = NelectronVander(torch.rand(3))

In [266]:
f(torch.tensor([1,2,3]))

tensor(-9.5996, grad_fn=<MulBackward0>)

In [267]:
f(torch.tensor([2,1,3]))

tensor(-9.5996, grad_fn=<MulBackward0>)